In [ ]:
%pip install feedparser
%pip install --upgrade openai
%pip install elevenlabs
%pip install python-telegram-bot --upgrade
%pip install nest_asyncio

In [ ]:
EPISODE_ID = 1 # Enter episode number for ID 
EPISODE_NAME = f"Pawdcast Episode {EPISODE_ID}"
EPISODE_TITLE = f"Pawdcast Episode {EPISODE_ID}.mp3"
EPISODE_TITLE_FULL = f"Pawdcast Episode {EPISODE_ID} Full.mp3"

In [ ]:
import requests
import feedparser
from openai import OpenAI
import os
import uuid
from elevenlabs import VoiceSettings
from elevenlabs.client import ElevenLabs
import moviepy.editor as mpe
from telegram import Bot, InputFile
import logging
import nest_asyncio

In [ ]:
FEED_URL = '.....' # Input RSS feed from URL
OPENAI_API_KEY = '......' # Enter your API key for OpenAI
ELEVENLABS_API_KEY = '......' # Enter your API key for ElevenLabs, you need to choose and add a voice to be used in your Voice Library in ElevenLabs.
ELEVENLABS_VOICE_ID = '.....' # Your choice of the voice with Voice ID from ElevenLabs

TOKEN = '.....' # Telegram bot token
CHAT_ID = '....' # Telegram chat id
MESSAGE_THREAD_ID = 1 # Telegram message thread id, you can find it on the telegram topic and taking the number after / in the share link


In [ ]:
d = feedparser.parse(FEED_URL)
client = OpenAI(api_key=OPENAI_API_KEY)
el_client = ElevenLabs(
    api_key=ELEVENLABS_API_KEY,
)

In [ ]:
d.entries[0].title

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You are a cat, who is hosting an podcast called Pawdcast for AI news, skilled in your cat things and presenting the current news in funny and engaging ways while still providing the important information."},
    {"role": "user", "content": d.entries[0].content[0].value}
  ]
)


In [ ]:
print(completion.choices[0].message.content)

In [ ]:
def text_to_speech_file(text: str) -> str:
    # Calling the text_to_speech conversion API with detailed parameters
    response = el_client.text_to_speech.convert(
        voice_id=ELEVENLABS_VOICE_ID, 
        optimize_streaming_latency="0",
        output_format="mp3_22050_32",
        text=text,
        model_id="eleven_turbo_v2", # use the turbo model for low latency, for other languages use the `eleven_multilingual_v2`
        voice_settings=VoiceSettings(
            stability=0.33,
            similarity_boost=0.59,
            style=0.0,
            use_speaker_boost=False,
        ),
    )

    # Writing the audio to a file
    with open(EPISODE_TITLE, "wb") as f:
        for chunk in response:
            if chunk:
                f.write(chunk)

    print(f"{EPISODE_NAME}: A new audio file was saved successfully!")


In [ ]:
text_to_speech_file(completion.choices[0].message.content)

In [ ]:
# Generate/find a intro music and put inside the same folder as the file

# Load the two audio files
meow_meow_meow = mpe.AudioFileClip("intro.mp3") 
pawdcast_episode = mpe.AudioFileClip(EPISODE_TITLE)

# Take the first 9.5 seconds of the meow meow meow song
meow_meow_meow_intro = meow_meow_meow.subclip(0, 9.5)

# Combine the intro with the Pawdcast episode
combined_audio = mpe.concatenate_audioclips([meow_meow_meow_intro, pawdcast_episode])

# Save the new file
combined_audio.write_audiofile(EPISODE_TITLE_FULL)


In [ ]:
nest_asyncio.apply()


async def post_pawdcast(bot):
    text_message = "Here's the latest AI Pawdcast episode! Enjoy and Meow out."

    # Sending message
    await bot.send_message(chat_id=CHAT_ID, text=text_message, message_thread_id=MESSAGE_THREAD_ID)
    # Sending audio
    with open(EPISODE_TITLE_FULL, 'rb') as audio_file:
        await bot.send_audio(chat_id=CHAT_ID, audio=audio_file, title=EPISODE_NAME, message_thread_id=MESSAGE_THREAD_ID)

async def main():
    # Initialize the bot
    bot = Bot(token=TOKEN)

    # Post the pawdcast to the specified chat
    await post_pawdcast(bot)

# Run the main function using asyncio
await main()

In [ ]:
# prompt: use the existing conversation and send another message that says 'add clickable links using markdown to the relevant text in the script, only reply with the script'

completion_script_links = client.chat.completions.create(
  model="gpt-4o",
  messages=[
    {"role": "system", "content": "You get a newsletter with links and a script without. Add links to the relevant parts in the script using markdown. Only reply with the script."},
    {"role": "user", "content": f"newsletter: {d.entries[0].content[0].value}, script: {completion.choices[0].message.content}"}
  ]
)


In [ ]:
async def post_message(message_part):
    bot = Bot(token=TOKEN)
    await bot.send_message(chat_id=CHAT_ID, text=message_part, message_thread_id=MESSAGE_THREAD_ID, parse_mode='Markdown', disable_web_page_preview=True)


In [ ]:
# prompt: split message into multiple messages using splitlines and join so that each message is roughly the same size and no larger than 4096 characters
message = completion_script_links.choices[0].message.content

lines = message.splitlines()
max_message_length = 4096
messages = []
current_message = ""

for line in lines:
    if len(current_message) + len(line) < max_message_length:
        current_message += line + "\n"
    else:
        messages.append(current_message)
        current_message = line + "\n"

if current_message:
    messages.append(current_message)

# Print the messages
for message in messages:
    await post_message(message)
